In [1]:
import yaml
import string
import requests

In [2]:
config_file = open('cocktails.yaml', 'r')
config = yaml.safe_load(config_file)

In [3]:
alphabet_string = string.ascii_lowercase
alphabet_list = list(alphabet_string)

In [4]:
cocktails_key = config['cocktails_key']

base_url = "https://www.thecocktaildb.com/api/"
json_api = 'json/v1/'
auth = f'{cocktails_key}'

request_bodies = []

for letter in alphabet_list:
    request_bodies.append(base_url + json_api + auth + f"/search.php?f={letter}")

In [1]:
#request_bodies

In [7]:
data = []

for request in request_bodies:
    data.append(requests.get(request).json())

In [8]:
drink = []

for letter in data:
    if letter['drinks'] != None: 
        for drinks in letter['drinks']:
            id= drinks['idDrink']
            dato=[id,drinks]
            drink_jaison = dato[2]
            drink_jaison["strInstructionsZH_HANS"] = drink_jaison["strInstructionsZH-HANS"]
            drink_jaison["strInstructionsZH_HANT"] = drink_jaison["strInstructionsZH-HANT"]
            del drink_jaison["strInstructionsZH-HANS"]
            del drink_jaison["strInstructionsZH-HANT"]
            dato[1] = drink_jaison
            drink.append(dato)

In [11]:
from google.cloud import storage

# bucket = config['bucket']
bucket = "cocktails-itam"

In [12]:
client = storage.Client()

In [13]:
gcs_bucket = client.get_bucket(bucket)

In [14]:
import json

In [22]:
for letter in drink:
    path = f"cocktails/{letter[0]}/ingredients.json"
    blob = gcs_bucket.blob(path)
    with blob.open(mode = 'w') as file:
        json.dump(letter[1], file)